In [53]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [54]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [55]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


In [56]:
# for data manipulation 
import pandas as pd
import numpy as np

# for EDA
import matplotlib.pyplot as plt
import seaborn as sns 
plt.figure(figsize=(12,7))
plt.style.use("seaborn-bright")

#for Stats

import statsmodels.api as sm
from statsmodels.api import Logit

# import libraries for data preprocessinhg
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

# libraries for model fitting
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# librarie for metrics 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_precision_recall_curve, plot_roc_curve, plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# libraries for optimization and interpretation
import shap

# for warnings
import warnings
warnings.filterwarnings("ignore")
print("All libraries are imported")

All libraries are imported


<Figure size 864x504 with 0 Axes>

In [57]:
data=pd.read_csv('Transformed.csv')
data.head()

,Unnamed: 0,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,...,Occupation_Crafts,Occupation_Homemaker,Occupation_Other,Occupation_Professional,Occupation_Retired,Occupation_Self,Occupation_Student,New_maritalstatus_Yes,CreditRating,Churn
0,0,-1.160580,-0.783685,-1.276518,-0.444774,-0.754292,-0.25613,-0.558645,-0.461939,-0.947937,...,0,0,0,1,0,0,0,0,3,1
1,1,-1.488361,-1.558803,-1.581047,-0.721787,-0.754292,-0.25613,0.029331,0.030061,-1.052171,...,0,0,0,1,0,0,0,1,2,1
2,2,-0.602404,-1.567202,-0.406192,-0.721787,-0.754292,-0.25613,0.037109,0.030061,-1.133754,...,1,0,0,0,0,0,0,1,2,0
3,3,0.406812,0.840653,0.957636,0.123980,-0.754292,-0.25613,0.662769,0.231433,0.956226,...,0,0,1,0,0,0,0,0,2,0
4,4,-1.481005,-1.600985,-1.581047,-0.721787,-0.754292,-0.25613,0.044890,0.025025,-1.133754,...,0,0,0,1,0,0,0,1,3,1


In [58]:
x=data.drop(columns=['Unnamed: 0','Churn'])

In [59]:
target=data['Churn']

### Logit summary

In [60]:
xc=sm.add_constant(x)

In [61]:
model=sm.Logit(target,xc)
result=model.fit(method='bfgs')
print(result.summary2())

         Current function value: 0.579284
         Iterations: 35
         Function evaluations: 37
         Gradient evaluations: 37
                                     Results: Logit
Model:                       Logit                   Pseudo R-squared:        0.032     
Dependent Variable:          Churn                   AIC:                     58835.0341
Date:                        2022-12-26 18:42        BIC:                     59365.0301
No. Observations:            50679                   Log-Likelihood:          -29358.   
Df Model:                    59                      LL-Null:                 -30341.   
Df Residuals:                50619                   LLR p-value:             0.0000    
Converged:                   0.0000                  Scale:                   1.0000    
----------------------------------------------------------------------------------------
                             Coef.    Std.Err.     z     P>|z|     [0.025       0.975]  
-------------

Observation:-
* More than half of features have p-val more than 0.05 hence they are not significant for the logistic regression model.

# fitting all the models

In [62]:
clf = {
    'logistic regression':LogisticRegression(),
    'KNN':KNeighborsClassifier(),
    'RandomForest':RandomForestClassifier(),
    'DecisionTree':DecisionTreeClassifier(),
    'XGBoost':XGBClassifier(),
    'AdaBoost':AdaBoostClassifier(),
    'NaiveBayes':GaussianNB()
}

model_score = pd.DataFrame(columns=['model name','precission score','recall score','acurracy score','f1 score'])

for clf_name,clf in list(zip(clf.keys(),clf.values())):
    print('fitting model .....',clf_name)
    clf.fit(x,target)
    y_pred=clf.predict(x)
    t={
        'model name':clf_name,
        'precission score': precision_score(target,y_pred),
        'recall score':recall_score(target,y_pred),
        'acurracy score':accuracy_score(target,y_pred),
        'f1 score':f1_score(target,y_pred),
        'roc_auc_score':roc_auc_score(target,y_pred)
    }
    model_score=model_score.append(t,ignore_index=True)
model_score=model_score.sort_values(by='roc_auc_score',ascending=False)
model_score

fitting model ..... logistic regression
fitting model ..... KNN
fitting model ..... RandomForest
fitting model ..... DecisionTree
fitting model ..... XGBoost
fitting model ..... AdaBoost
fitting model ..... NaiveBayes


,model name,precission score,recall score,acurracy score,f1 score,roc_auc_score
2,RandomForest,1.0,1.0,1.0,1.0,1.000000
3,DecisionTree,1.0,1.0,1.0,1.0,1.000000
4,XGBoost,0.879937,0.385575,0.809112,0.536197,0.682242
1,KNN,0.686383,0.352065,0.768543,0.465409,0.643787
6,NaiveBayes,0.387898,0.198028,0.681071,0.262199,0.536376
5,AdaBoost,0.559045,0.09205,0.719391,0.158072,0.531471
0,logistic regression,0.503885,0.031304,0.713964,0.058946,0.509474


In [63]:
 from sklearn.model_selection import cross_val_score,KFold

In [64]:
kf=KFold(n_splits=5,shuffle=True, random_state=0)

In [65]:

clf = {
    'logistic regression':LogisticRegression(),
    'KNN':KNeighborsClassifier(),
    'RandomForest':RandomForestClassifier(),
    'DecisionTree':DecisionTreeClassifier(),
    'XGBoost':XGBClassifier(),
    'AdaBoost':AdaBoostClassifier(),
    'NaiveBayes':GaussianNB() 
}

model_score = pd.DataFrame(columns=['model name','recall scores','recall score','variance','roc_auc scores','roc_auc','roc_auc_variance'])

for clf_name,clf in list(zip(clf.keys(),clf.values())):
    print('fitting model .....',clf_name)
    clf.fit(x,target)
    cv=cross_val_score(estimator=clf,X=x,y=target,cv=kf,scoring='recall')
    cv_auc=cross_val_score(estimator=clf,X=x,y=target,cv=kf,scoring='roc_auc')
    t={
        'model name':clf_name,
        'recall scores':cv,
        'recall score':np.mean(cv ),
        'variance':np.std(cv,ddof=1),
        'roc_auc scores':cv_auc,
        'roc_auc':np.mean(cv_auc),
        'roc_auc_variance':np.std(cv_auc,ddof=1)
    }
    model_score=model_score.append(t,ignore_index=True)
model_score=model_score.sort_values(by='recall score',ascending=False)
model_score
    

fitting model ..... logistic regression
fitting model ..... KNN
fitting model ..... RandomForest
fitting model ..... DecisionTree
fitting model ..... XGBoost
fitting model ..... AdaBoost
fitting model ..... NaiveBayes


,model name,recall scores,recall score,variance,roc_auc scores,roc_auc,roc_auc_variance
3,DecisionTree,"[0.34395120298203996, 0.3817427385892116, 0.37...",0.361098,0.016756,"[0.5391414914963424, 0.5467440789458611, 0.550...",0.544344,0.00462
6,NaiveBayes,"[0.20230430362588953, 0.2053941908713693, 0.18...",0.19671,0.009132,"[0.5748554150639993, 0.5811292151543548, 0.580...",0.578359,0.003752
4,XGBoost,"[0.19688241274144358, 0.19674965421853388, 0.1...",0.19154,0.005077,"[0.6595771057167322, 0.6639770988037603, 0.662...",0.663247,0.003408
1,KNN,"[0.17417824466282616, 0.1718533886583679, 0.16...",0.176415,0.009775,"[0.5562975361665732, 0.5570310536959857, 0.558...",0.559556,0.005148
5,AdaBoost,"[0.09251101321585903, 0.09301521438450899, 0.0...",0.092019,0.005577,"[0.6535708381882037, 0.6625686980516331, 0.660...",0.660315,0.003881
2,RandomForest,"[0.082006099627245, 0.06811894882434301, 0.073...",0.077249,0.006599,"[0.6592241828784553, 0.6609296490327665, 0.653...",0.658275,0.003291
0,logistic regression,"[0.027787190782785496, 0.030428769017980636, 0...",0.031478,0.003193,"[0.6231112815277696, 0.625555856594822, 0.6177...",0.621595,0.002985


## checking with the non-transformed data

In [66]:
data2=pd.read_csv('finaldata.csv')

In [67]:
data2.head()

,Unnamed: 0,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,...,Occupation_Crafts,Occupation_Homemaker,Occupation_Other,Occupation_Professional,Occupation_Retired,Occupation_Self,Occupation_Student,New_maritalstatus_Yes,CreditRating,Churn
0,0,-0.783843,-0.580074,-1.045645,-0.289834,-0.414232,-0.125819,-0.564829,-0.449982,-0.588732,...,0,0,0,1,0,0,0,0,3,1
1,1,-0.941334,-0.974398,-1.255508,-0.401835,-0.414232,-0.125819,0.029311,0.030120,-0.632953,...,0,0,0,1,0,0,0,1,2,1
2,2,-0.469311,-0.978171,-0.374082,-0.401835,-0.414232,-0.125819,0.037078,0.030120,-0.666120,...,1,0,0,0,0,0,0,1,2,0
3,3,0.525509,1.482105,1.178906,0.153688,-0.414232,-0.125819,0.654518,0.234795,5.082715,...,0,0,1,0,0,0,0,0,2,0
4,4,-0.937964,-0.993265,-1.255508,-0.401835,-0.414232,-0.125819,0.044845,0.025067,-0.666120,...,0,0,0,1,0,0,0,1,3,1


In [68]:
target=data2['Churn']

In [69]:
ind=data2.drop(columns=['Unnamed: 0','Churn'])

In [70]:

clf = {
    'logistic regression':LogisticRegression(),
    'KNN':KNeighborsClassifier(),
    'RandomForest':RandomForestClassifier(),
    'DecisionTree':DecisionTreeClassifier(),
    'XGBoost':XGBClassifier(),
    'AdaBoost':AdaBoostClassifier(),
    'NaiveBayes':GaussianNB()
}

model_score = pd.DataFrame(columns=['model name','recall scores','recall score','variance','roc_auc scores','roc_auc','roc_auc_variance'])

for clf_name,clf in list(zip(clf.keys(),clf.values())):
    print('fitting model .....',clf_name)
    clf.fit(ind,target)
    cv=cross_val_score(estimator=clf,X=ind,y=target,cv=kf,scoring='recall')
    cv_auc=cross_val_score(estimator=clf,X=ind,y=target,cv=kf,scoring='roc_auc')
    t={
        'model name':clf_name,
        'recall scores':cv,
        'recall score':np.mean(cv ),
        'variance':np.std(cv,ddof=1),
        'roc_auc scores':cv_auc,
        'roc_auc':np.mean(cv_auc),
        'roc_auc_variance':np.std(cv_auc,ddof=1)
    }
    model_score=model_score.append(t,ignore_index=True)
model_score=model_score.sort_values(by='recall score',ascending=False)
model_score
    

fitting model ..... logistic regression
fitting model ..... KNN
fitting model ..... RandomForest
fitting model ..... DecisionTree
fitting model ..... XGBoost
fitting model ..... AdaBoost
fitting model ..... NaiveBayes


,model name,recall scores,recall score,variance,roc_auc scores,roc_auc,roc_auc_variance
6,NaiveBayes,"[0.6038630972551677, 0.27835408022130015, 0.26...",0.416878,0.182308,"[0.5624113595594195, 0.5675354784004009, 0.568...",0.566196,0.003442
3,DecisionTree,"[0.3476787529650966, 0.3724066390041494, 0.368...",0.358846,0.010823,"[0.5406270641305083, 0.5467762513241273, 0.544...",0.540875,0.004903
4,XGBoost,"[0.19789901728227718, 0.19951590594744123, 0.1...",0.19133,0.009316,"[0.6553106209116804, 0.6671958641023467, 0.660...",0.661141,0.00445
1,KNN,"[0.18027787190782785, 0.1780774550484094, 0.18...",0.181457,0.003159,"[0.5531180706821957, 0.5485637276578584, 0.557...",0.554161,0.003614
5,AdaBoost,"[0.09013893595391392, 0.09301521438450899, 0.0...",0.091545,0.005626,"[0.6530437413499594, 0.6625686980516331, 0.660...",0.66021,0.004111
2,RandomForest,"[0.08065062690613352, 0.0698478561549101, 0.07...",0.07657,0.005336,"[0.6556558089717297, 0.6574326929025252, 0.660...",0.658602,0.002186
0,logistic regression,"[0.023720772619451033, 0.03008298755186722, 0....",0.02786,0.004717,"[0.6140955014011032, 0.6187608020908036, 0.614...",0.614157,0.003442
